In [1]:
from PIL import Image, ImageDraw
import os
from zipfile import ZipFile

In [2]:
# Dataset 위치
dataset_path = 'Dataset/'
zip_list = os.listdir(dataset_path)
zip_list

['19062421',
 '19062421.zip',
 '19062431',
 '19062431.zip',
 '19062521',
 '19062521.zip',
 '19062531',
 '19062531.zip',
 '19062542',
 '19062542.zip',
 '19062621',
 '19062621.zip',
 '19062622',
 '19062622.zip',
 '19062641',
 '19062641.zip',
 '19062722',
 '19062722.zip',
 '19062731',
 '19062731.zip',
 '19062732',
 '19062732.zip',
 '19062811',
 '19062811.zip',
 '19062831',
 '19062831.zip',
 '19062832',
 '19062832.zip',
 '19062841',
 '19062841.zip',
 '19062842',
 '19062842.zip',
 '19070121',
 '19070121.zip',
 '19070142',
 '19070142.zip',
 '19070212',
 '19070212.zip',
 '19070221',
 '19070221.zip',
 '19070222',
 '19070222.zip',
 '19070231',
 '19070231.zip',
 '19070241',
 '19070241.zip',
 '19070242',
 '19070242.zip',
 '19070311',
 '19070311.zip',
 '19070312',
 '19070312.zip',
 '19070321',
 '19070321.zip',
 '19070322',
 '19070322.zip',
 '19070331',
 '19070331.zip',
 '19070412',
 '19070412.zip',
 '19070421',
 '19070421.zip',
 '19070431',
 '19070431.zip',
 '19070442',
 '19070442.zip',
 '19070522

In [3]:
# unzip dataset
for z in zip_list:
    if z == '.ipynb_checkpoints':
        continue
    with ZipFile(dataset_path + z, 'r') as zip:
        print(z + ': unzip 시작')
        zip.extractall(dataset_path + z.replace('.zip' ,''))
        print(z + ': unzip 완료')
print('unzip 종료')

PermissionError: [Errno 13] Permission denied: 'Dataset/19062421'

In [4]:
# 전처리한 데이터를 저장할 디렉토리 생성
def createDir(dir_name):
    try:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    except OSError:
        print('Error: 디렉터리 생성오류. 디렉토리명: ' + dir_name)

In [5]:
# 사용할 사진 경로
accessory = 'S001'  # 보통, 안경
light = 'L1'
emotion = ('E01', 'E02')      # 무표정, 웃음
camera = ('C6', 'C7', 'C16', 'C17', 'C19')

In [6]:
# bbox 그리기
def show_bbox(img_path, bbox):

    img = Image.open(dataset_path + img_path)
    draw = ImageDraw.Draw(img)
    bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
    draw.rectangle(bbox, outline=(0, 255, 0), width=1)

    img.show()

In [6]:
# bbox 정보 구조화
# 코 끝(0), 우안(1), 좌안(2), 우측 입꼬리(3), 좌측 입꼬리(4), 우측 귀(5), 좌측 귀(6)
def bbox_info(txt_path):
    
    info = {}
    
    with open(dataset_path + txt_path , 'r') as f:
        
        # 좌안, 우안 위치 확인
        for i in range(0, 7):
            coordinate = f.readline()
            if i == 5:
                info['right_ear'] = tuple(map(int, coordinate.replace('\n', '').split('\t')))
            elif i == 6:
                info['left_ear'] = tuple(map(int, coordinate.replace('\n', '').split('\t')))
        
        # 얼굴 bbox
        face_bbox = f.readline()
        info['face_bbox'] = tuple(map(int, face_bbox.replace('\n', '').split('\t')))
        
        while True:
            
            bbox = f.readline()
            
            if bbox == '':
                break
                
    return info

In [35]:
# face_bbox 정보 수정
def resize_bbox_1(info):
    resized_bbox = list(info['face_bbox'])
    resized_bbox[0] = info['right_ear'][0]   # start_x
    resized_bbox[1] = info['face_bbox'][1]   # start_y
    resized_bbox[2] = info['left_ear'][0]    # start_x + width
    resized_bbox[3] = info['face_bbox'][1] + info['face_bbox'][3]  # start_y + height
    
    return resized_bbox

def resize_bbox(info, camera):
    result = resize_bbox_1(info);
    
    if camera == 'C6':
        result[0] = result[0] - 2
        result[2] = result[2] + 1
    elif camera == 'C17':
        result[2] = result[2] + 5
    elif camera ==  'C19':
        result[0] = result[0] - 7

    return result
            

In [29]:
# test
info = bbox_info('/19062421/S001/L1/E01/C6.txt')
print(info)
resized_bbox = resize_bbox(info, 'C6')
print(resized_bbox)
# show_bbox('/19062421/S001/L1/E01/C4.jpg', resized_bbox)

{'right_ear': (77, 59), 'left_ear': (109, 57), 'face_bbox': (65, 24, 58, 58)}
[67, 24, 119, 82]


In [ ]:
# crop image
processed_path = "ProcessedDataset/"
for z in zip_list:
    face_id = z.replace('.zip', '')
    i = 0
    if face_id == "19082622": continue
    for e in emotion:
        for c in camera:
            path = '/{face_id}/{accessory}/{light}/{emotion}/{camera}'.format(
                face_id=face_id, 
                accessory=accessory,
                light=light,
                emotion=e,
                camera=c
            )
            createDir(processed_path + face_id)
            
            img_path = path + '.jpg'
            txt_path = path + '.txt'
            # bbox에서 양쪽 귀 좌표 추출
            info = bbox_info(txt_path)
            resized_bbox = resize_bbox(info, c)
            print(resized_bbox)

            # crop
            img = Image.open(dataset_path +img_path)
            print(img.size)
            cropped_img = img.crop(tuple(resized_bbox))
            cropped_img_path = processed_path + face_id +'/' + str(i) + '.jpg'
            print(cropped_img_path)
            cropped_img.save(cropped_img_path)
            i += 1



[75, 24, 110, 82]
(173, 115)
ProcessedDataset/19062421/0.jpg
[72, 26, 105, 83]
(173, 115)
ProcessedDataset/19062421/1.jpg
[74, 25, 105, 78]
(173, 115)
ProcessedDataset/19062421/2.jpg
[69, 18, 101, 73]
(173, 115)
ProcessedDataset/19062421/3.jpg
[77, 30, 115, 84]
(173, 115)
ProcessedDataset/19062421/4.jpg
[79, 25, 112, 82]
(173, 115)
ProcessedDataset/19062421/5.jpg
[72, 26, 105, 81]
(173, 115)
ProcessedDataset/19062421/6.jpg
[74, 24, 105, 77]
(173, 115)
ProcessedDataset/19062421/7.jpg
[69, 22, 102, 74]
(173, 115)
ProcessedDataset/19062421/8.jpg
[76, 31, 115, 80]
(173, 115)
ProcessedDataset/19062421/9.jpg
[75, 24, 110, 82]
(173, 115)
ProcessedDataset/19062421/0.jpg
[72, 26, 105, 83]
(173, 115)
ProcessedDataset/19062421/1.jpg
[74, 25, 105, 78]
(173, 115)
ProcessedDataset/19062421/2.jpg
[69, 18, 101, 73]
(173, 115)
ProcessedDataset/19062421/3.jpg
[77, 30, 115, 84]
(173, 115)
ProcessedDataset/19062421/4.jpg
[79, 25, 112, 82]
(173, 115)
ProcessedDataset/19062421/5.jpg
[72, 26, 105, 81]
(173, 